In [3]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, date 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import TimeSeriesSplit


# 1. Reading the data collection 

In [4]:
total_data = pd.read_csv("../input/acea-water-prediction/Aquifer_Petrignano.csv")
total_data.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '../input/acea-water-prediction/Aquifer_Petrignano.csv'

# 2. Cleaning the dataframe

In [ ]:
#Remove columns with Nan values and others we won't use
total_data = total_data[total_data.Rainfall_Bastia_Umbra.notna()].reset_index(drop=True)
total_data = total_data.drop(['Depth_to_Groundwater_P24', 'Temperature_Petrignano'], axis=1)

In [ ]:
#Lowering the names of the columns
total_data.columns = ['date', 'rainfall', 'depth_to_groundwater', 'temperature', 'drainage_volume', 'river_hydrometry']
target = ['depth_to_groundwater']
values = [value for value in total_data.columns if value not in target]
total_data.head(5)

In [ ]:
#Since we are working with a time series, we need to transfor the dataframe
total_data['date'] = pd.to_datetime(total_data['date'], format = '%d/%m/%Y')
new_data = total_data.set_index('date')
new_data.head(5)

# 3. Visualization of the information

In [ ]:
fig, ax = plt.subplots(5, 1, figsize = (14, 24))
fig.suptitle("Evolution", fontsize=16)

for i, column in enumerate(total_data.drop('date', axis=1).columns):
    sns.lineplot(x=total_data['date'], y=total_data[column].fillna(method='ffill'), ax=ax[i], color='orange')
    ax[i].set_title('Caracteristic: {}'.format(column), fontsize=14)
    ax[i].set_ylabel(ylabel=column, fontsize=14)

plt.tight_layout()
plt.show()

# 4. Processing the data

In [ ]:
total_data = total_data.sort_values(by='date')
total_data['delta'] = total_data['date'] - total_data['date'].shift(1)
total_data[['date', 'delta']].head()

In [ ]:
total_data['delta'].sum(), total_data['delta'].count()

In [ ]:
total_data = total_data.drop('delta', axis=1)
total_data.isna().sum()

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(15, 15))

pre_hydrometry = total_data['river_hydrometry'].copy()
total_data['river_hydrometry'] = total_data['river_hydrometry'].replace(0, np.nan)

sns.lineplot(x=total_data['date'], y=pre_hydrometry, ax=ax[0], color='blue', label='original')
sns.lineplot(x=total_data['date'], y=total_data['river_hydrometry'].fillna(np.inf), ax=ax[0], color='orange', label='modified')
ax[0].set_title('Characteristic: Hydrometry', fontsize=14)
ax[0].set_ylabel(ylabel='Hydrometry', fontsize=14)
ax[0].set_xlim([date(2009, 1, 1), date(2020, 6, 30)])

pre_drainage = total_data['drainage_volume'].copy()
total_data['drainage_volume'] = total_data['drainage_volume'].replace(0, np.nan)

sns.lineplot(x=total_data['date'], y=pre_drainage, ax=ax[1], color='green', label='original')
sns.lineplot(x=total_data['date'], y=total_data['drainage_volume'].fillna(np.inf), ax=ax[1], color='orange', label='modified')
ax[1].set_title('Characteristic: Drainage', fontsize=14)
ax[1].set_ylabel(ylabel='Drainage', fontsize=14)
ax[1].set_xlim([date(2009, 1, 1), date(2020, 6, 30)])

In [ ]:
#For the missing values, we'll interpolate them
total_data['drainage_volume'] = total_data['drainage_volume'].interpolate()
total_data['river_hydrometry'] = total_data['river_hydrometry'].interpolate()
total_data['depth_to_groundwater'] = total_data['depth_to_groundwater'].interpolate()

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(18, 15)) 

# First Column: Drainage Volume
sns.lineplot(x='date', y='drainage_volume', data=total_data, color='blue', ax=ax[0, 0])
ax[0, 0].set_title('Drainage', fontsize=14)

# Weekly Drainage
resampled_weekly = total_data[['date', 'drainage_volume']].resample('7D', on='date').sum().reset_index()
sns.lineplot(x='date', y='drainage_volume', data=resampled_weekly, color='orange', ax=ax[1, 0])
ax[1, 0].set_title('Weekly Drainage', fontsize=14)

# Monthly Drainage
resampled_monthly = total_data[['date', 'drainage_volume']].resample('M', on='date').sum().reset_index()
sns.lineplot(x='date', y='drainage_volume', data=resampled_monthly, color='green', ax=ax[0, 1])  # Changed to fit within the grid
ax[0, 1].set_title('Monthly Drainage', fontsize=14)

# Adjust date range for the first column
for i in range(2):  # Loop over the first column (ax[0, 0] and ax[1, 0])
    ax[i, 0].set_xlim([date(2009, 1, 1), date(2020, 6, 30)])

# Second Column: Temperature
sns.lineplot(x='date', y='temperature', data=total_data, color='pink', ax=ax[1, 1])  # Corrected placement
ax[1, 1].set_title('Daily Temperature', fontsize=14)

# Weekly Temperature
resampled_temp_weekly = total_data[['date', 'temperature']].resample('7D', on='date').mean().reset_index()
sns.lineplot(x='date', y='temperature', data=resampled_temp_weekly, color='red', ax=ax[0, 2])  # Adjusted grid placement
ax[0, 2].set_title('Weekly Temperature', fontsize=14)

# Monthly Temperature
resampled_temp_monthly = total_data[['date', 'temperature']].resample('M', on='date').mean().reset_index()
sns.lineplot(x='date', y='temperature', data=resampled_temp_monthly, color='yellow', ax=ax[1, 2])  # Adjusted grid placement
ax[1, 2].set_title('Monthly Temperature', fontsize=14)

# Adjust date range for the second column
for i in range(2):  # Loop over the second column (ax[0, 1] and ax[1, 1])
    ax[i, 1].set_xlim([date(2009, 1, 1), date(2020, 6, 30)])

plt.tight_layout()
plt.show()

In [ ]:
downsample = total_data[['date',
                 'depth_to_groundwater', 
                 'temperature',
                 'drainage_volume', 
                 'river_hydrometry',
                 'rainfall'
                ]].resample('7D', on='date').mean().reset_index(drop=False)

total_data = downsample.copy()

In [ ]:
rolling_window = 52
f, ax = plt.subplots(2, 1, figsize=(15, 12))

sns.lineplot(x=total_data['date'], y=total_data['drainage_volume'], ax=ax[0], color='blue')
sns.lineplot(x=total_data['date'], y=total_data['drainage_volume'].rolling(rolling_window).mean(), ax=ax[0], color='black', label='rolling mean')
sns.lineplot(x=total_data['date'], y=total_data['drainage_volume'].rolling(rolling_window).std(), ax=ax[0], color='orange', label='rolling std')
ax[0].set_title('Depth to Groundwater', fontsize=14)
ax[0].set_ylabel(ylabel='Drainage Volume', fontsize=14)
ax[0].set_xlim([date(2009, 1, 1), date(2020, 6, 30)])

sns.lineplot(x=total_data['date'], y=total_data['temperature'], ax=ax[1], color='pink')
sns.lineplot(x=total_data['date'], y=total_data['temperature'].rolling(rolling_window).mean(), ax=ax[1], color='black', label='rolling mean')
sns.lineplot(x=total_data['date'], y=total_data['temperature'].rolling(rolling_window).std(), ax=ax[1], color='orange', label='rolling std')
ax[1].set_title('Temperature', fontsize=14)
ax[1].set_ylabel(ylabel='Temperature', fontsize=14)
ax[1].set_xlim([date(2009, 1, 1), date(2020, 6, 30)])

plt.tight_layout()
plt.show()

In [ ]:
res = adfuller(total_data['depth_to_groundwater'].values)
print("Dickey-Fuller test results:")
res

In [ ]:
f, ax = plt.subplots(nrows=3, ncols=2, figsize=(15, 9))

def visualize_adfuller(series, title, ax):
    result = adfuller(series)
    significance_level = 0.05
    adf_stat = result[0]
    p_val = result[1]
    crit_val_1 = result[4]['1%']
    crit_val_5 = result[4]['5%']
    crit_val_10 = result[4]['10%']

    if (p_val < significance_level) & ((adf_stat < crit_val_1)):
        linecolor = 'orange' 
    elif (p_val < significance_level) & (adf_stat < crit_val_5):
        linecolor = 'pink'
    elif (p_val < significance_level) & (adf_stat < crit_val_10):
        linecolor = 'blue'
    else:
        linecolor = 'red'
    sns.lineplot(x=total_data['date'], y=series, ax=ax, color=linecolor)
    ax.set_title(f'ADFuller Statistic {adf_stat:0.3f}, p-value: {p_val:0.3f}\nCritical Values 1%: {crit_val_1:0.3f}, 5%: {crit_val_5:0.3f}, 10%: {crit_val_10:0.3f}', fontsize=14)
    ax.set_ylabel(ylabel=title, fontsize=14)

visualize_adfuller(total_data['rainfall'].values, 'Rainfall', ax[0, 0])
visualize_adfuller(total_data['temperature'].values, 'Temperature', ax[1, 0])
visualize_adfuller(total_data['river_hydrometry'].values, 'River_Hydrometry', ax[0, 1])
visualize_adfuller(total_data['drainage_volume'].values, 'Drainage_Volume', ax[1, 1])
visualize_adfuller(total_data['depth_to_groundwater'].values, 'Depth_to_Groundwater', ax[2, 0])

f.delaxes(ax[2, 1])
plt.tight_layout()
plt.show()

In [ ]:
total_data['depth_to_groundwater_log'] = np.log(abs(total_data['depth_to_groundwater']))

f, ax = plt.subplots(1, 2, figsize=(18, 6))
visualize_adfuller(total_data['depth_to_groundwater_log'], 'Transformed \n Depth to Groundwater', ax[0])

sns.distplot(total_data['depth_to_groundwater_log'], ax=ax[1])

In [ ]:
diff = np.diff(total_data['depth_to_groundwater'])
total_data['depth_to_groundwater_diff_1'] = np.append([0], diff)

f, ax = plt.subplots(1, 1, figsize=(18, 6))
visualize_adfuller(total_data['depth_to_groundwater_diff_1'], 'Differenced \n Depth to Groundwater', ax)

# 5. Feature engineering

In [ ]:
total_data['year'] = pd.DatetimeIndex(total_data['date']).year
total_data['month'] = pd.DatetimeIndex(total_data['date']).month
total_data['day'] = pd.DatetimeIndex(total_data['date']).day
total_data['day_of_year'] = pd.DatetimeIndex(total_data['date']).dayofyear
total_data['week_of_year'] = pd.DatetimeIndex(total_data['date']).isocalendar().week
total_data['quarter'] = pd.DatetimeIndex(total_data['date']).quarter
total_data['season'] = total_data['month'] % 12 // 3 + 1

total_data[['date', 'year', 'month', 'day', 'day_of_year', 'week_of_year', 'quarter', 'season']].head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 5))

sns.lineplot(x=total_data['date'], y=total_data['month'], color='orange')
ax.set_xlim([date(2009, 1, 1), date(2020, 6, 30)])
plt.show()

In [ ]:
month_in_year = 12
total_data['month_sin'] = np.sin(2*np.pi*total_data['month']/month_in_year)
total_data['month_cos'] = np.cos(2*np.pi*total_data['month']/month_in_year)

fig, ax = plt.subplots(1, 1, figsize=(7, 7))

sns.scatterplot(x=total_data.month_sin, y=total_data.month_cos, color='orange')
plt.show()

In [ ]:
main_columns =  [
    'rainfall', 'temperature', 'drainage_volume', 
    'river_hydrometry', 'depth_to_groundwater'
]

for column in main_columns:
    decomp = seasonal_decompose(total_data[column], period=52, model='additive', extrapolate_trend='freq')
    total_data[f"{column}_trend"] = decomp.trend
    total_data[f"{column}_seasonal"] = decomp.seasonal

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(16, 8))

for i, column in enumerate(['temperature', 'depth_to_groundwater']):
    ts_data = total_data[column].dropna()

    res = seasonal_decompose(ts_data, period=52, model='additive', extrapolate_trend='freq')

    ax[0, i].set_title(f'Decomposition of {column}', fontsize=14)
    res.observed.plot(ax=ax[0, i], legend=False, color='orange')
    res.trend.plot(ax=ax[1, i], legend=False, color='blue')
    res.seasonal.plot(ax=ax[2, i], legend=False, color='green')
    res.resid.plot(ax=ax[3, i], legend=False, color='red')

plt.tight_layout()
plt.show()

In [ ]:
weeks_in_month = 4

for column in main_columns:
    total_data[f'{column}_seasonal_shift_b_2m'] = total_data[f'{column}_seasonal'].shift(-2 * weeks_in_month)
    total_data[f'{column}_seasonal_shift_b_1m'] = total_data[f'{column}_seasonal'].shift(-1 * weeks_in_month)
    total_data[f'{column}_seasonal_shift_1m'] = total_data[f'{column}_seasonal'].shift(1 * weeks_in_month)
    total_data[f'{column}_seasonal_shift_2m'] = total_data[f'{column}_seasonal'].shift(2 * weeks_in_month)
    total_data[f'{column}_seasonal_shift_3m'] = total_data[f'{column}_seasonal'].shift(3 * weeks_in_month)

# 6. EDA analysis

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(15, 12))
f.suptitle('Seasonal Components of Features', fontsize=16)

for i, column in enumerate(main_columns):
    sns.lineplot(x=total_data['date'], y=total_data[column + '_seasonal'], ax=ax[i], color='orange', label='P25')
    ax[i].set_ylabel(ylabel=column, fontsize=14)
    ax[i].set_xlim([date(2017, 9, 30), date(2020, 6, 30)])
    
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 9))

matrix = total_data[main_columns].corr()

sns.heatmap(matrix, annot=True, vmin=-1, vmax=1, cmap='coolwarm_r', ax=ax[0])
ax[0].set_title('Correlation Matrix - Main Characterictics', fontsize=16)

changed_cols = [
    'depth_to_groundwater_seasonal',         
    'temperature_seasonal_shift_b_2m',
    'drainage_volume_seasonal_shift_2m', 
    'river_hydrometry_seasonal_shift_3m'
]
matrix = total_data[changed_cols].corr()

sns.heatmap(matrix, annot=True, vmin=-1, vmax=1, cmap='coolwarm_r', ax=ax[1])
ax[1].set_title('Correlation Matrix', fontsize=16)


plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(18, 9))

plot_acf(total_data['depth_to_groundwater_diff_1'], lags=100, ax=ax[0])
plot_pacf(total_data['depth_to_groundwater_diff_1'], lags=100, ax=ax[1])

plt.show()

In [ ]:
n_spl = 3

X = total_data['date']
y = total_data['depth_to_groundwater']

folds = TimeSeriesSplit(n_splits=n_spl)

In [ ]:
fig, ax = plt.subplots(n_spl, 2, figsize=(18, 9))

for i, (train_index, valid_index) in enumerate(folds.split(X)):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    sns.lineplot(
        x=X_train, 
        y=y_train, 
        ax=ax[i,0], 
        color='orange', 
        label='train'
    )
    sns.lineplot(
        x=X_train[len(X_train) - len(X_valid):(len(X_train) - len(X_valid) + len(X_valid))], 
        y=y_train[len(X_train) - len(X_valid):(len(X_train) - len(X_valid) + len(X_valid))], 
        ax=ax[i,1], 
        color='red', 
        label='train'
    )

    for j in range(2):
        sns.lineplot(x= X_valid, y= y_valid, ax=ax[i, j], color='green', label='validation')
    ax[i, 0].set_title(f"Rolling Window with Adjusting Training Size (Split {i+1})", fontsize=16)
    ax[i, 1].set_title(f"Rolling Window with Constant Training Size (Split {i+1})", fontsize=16)

for i in range(n_spl):
    ax[i, 0].set_xlim([date(2009, 1, 1), date(2020, 6, 30)])
    ax[i, 1].set_xlim([date(2009, 1, 1), date(2020, 6, 30)])
    
plt.tight_layout()
plt.show()

In [ ]:
train_size = int(0.85 * len(df))
test_size = len(df) - train_size

univar = total_data[['date', 'depth_to_groundwater']].copy()
univar.columns = ['ds', 'y']

train = univar.iloc[:train_size, :]

x_train, y_train = pd.DataFrame(univar.iloc[:train_size, 0]), pd.DataFrame(univar.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(univar.iloc[train_size:, 0]), pd.DataFrame(univar.iloc[train_size:, 1])

print(len(train), len(x_valid))

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima import auto_arima

model = auto_arima(total_data, seasonal = False, trace = True)

In [ ]:
forecast = model.predict(60)
forecast

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))
fig.suptitle("Future predictions", fontsize=16)

sns.lineplot(data = total_data, color = "red")
sns.lineplot(data = forecast, c = "blue")

plt.tight_layout()
plt.show()